In [1]:
import numpy as np
import pandas as pd
import import_ipynb

In [ ]:
import Orders, Freights
import Customers, PlantPorts, ProductsPerPlant, WhCapacities, WhCosts

# 2da Etapa: Optimización

*Función Objetivo*\
Se desea conocer un conjunto de plantas de almacenaje, puertos de origen y puertos de destino que minimicen el costo total de la cadena de suministro.

Nomenclatura:
- k: ID de pedido
- i: ID de planta de almacenamiento
- p: ID de puerto de origen
- j: ID de puerto de destino
- c: ID de transportista
- s: Nivel de servicio
- t: Tiempo de envío [días]
- m: Modo de transporte (aire o tierra)
- q: Cantidad de items en cada pedido [items]
- w: Peso [kg/item]
- F: Peso máximo [kg/item]

- P: Costo de almacenamiento [USD/item]
- M: Costo fijo de transporte [USD/kg]
- X: Costo de almacenamiento [USD]
- Y: Costo de transporte [USD]
- Z: Costo total de la cadena de suministro [USD]

### Costos del Almacenamiento



$$ X_{i} = \sum_{k}{(q_{k}*P_i)} $$

In [3]:
df1 = pd.merge(Orders.pedidos[["Planta", "Items/pedido"]], WhCosts.costos_almacenaje["Costo [USD/item]"], how = "cross")
df1 = pd.merge(df1, PlantPorts.puertos_por_planta, on = "Planta")

df1["Costo [USD/pedido]"] = df1["Items/pedido"] * df1["Costo [USD/item]"]

g1 = df1.groupby(by ="Planta").agg({"Costo [USD/pedido]": np.sum})
g1

,Costo [USD/pedido]
Planta,
3,2.498690e+08
4,3.039130e+03
8,2.419147e+06
9,1.954737e+06
12,9.781107e+05
13,3.788345e+05


### Capacidad del Almacenamiento

$$ \sum_{k}{o_{ki}} \leq \ C_{i} $$

In [19]:
df2 = pd.merge(Orders.pedidos[["Pedido", "Planta"]], WhCapacities.capacidad_almacenaje, on = "Planta")
df2 = pd.merge(df2, PlantPorts.puertos_por_planta, on = ["Planta"])
df2 = pd.merge(df2, ProductsPerPlant.productos_por_planta, on = ["Planta"])

df2["Recuento de Pedidos"] = df2.groupby(by = ["Planta"])["Pedido"].transform("sum")
df2["Recuento de Pedidos"] = np.where(df2["Recuento de Pedidos"] <= df2["Capacidad [pedidos/día]"], df2["Recuento de Pedidos"], df2["Capacidad [pedidos/día]"])

df2.set_index("Planta").sample(10)

,Pedido,Capacidad [pedidos/día],Puerto de origen,Producto,Recuento de Pedidos
Planta,,,,,
3,1447361718,1013,4,1689551,1013.0
3,1447283683,1013,4,1668608,1013.0
3,1447367882,1013,4,1669701,1013.0
3,1447410132,1013,4,1680742,1013.0
3,1447286365,1013,4,1686401,1013.0
3,1447300219,1013,4,1698988,1013.0
3,1447283839,1013,4,1690632,1013.0
3,1447396243,1013,4,1682814,1013.0
3,1447198883,1013,4,1699555,1013.0


### Costos de Transporte

In [5]:
df3 = pd.merge(Orders.pedidos, Freights.fletes, on = ["Puerto de origen", "Puerto de destino", "Transportista", "Nivel de servicio"])
df3 = pd.merge(df3, PlantPorts.puertos_por_planta, on = ["Planta", "Puerto de origen"])
df3 = pd.merge(df3, ProductsPerPlant.productos_por_planta, on = ["Planta", "Producto"])

df3[["Puerto de origen", "Puerto de destino", "Transportista", "Nivel de servicio", "Tasa de flete [USD/kg]"]].set_index(["Puerto de origen", "Puerto de destino"]).sample(10).round(2)

Transportista Nivel de servicio  \
Puerto de origen Puerto de destino                                   
4                9                        V444_1               DTD   
                 9                        V444_1               DTD   
                 9                        V444_0               DTP   
                 9                        V444_1               DTD   
                 9                        V444_0               DTP   
                 9                        V444_1               DTD   
                 9                        V444_0               DTP   
                 9                        V444_0               DTP   
                 9                        V444_0               DTP   
                 9                        V444_0               DTP   

                                    Tasa de flete [USD/kg]  
Puerto de origen Puerto de destino                          
4                9                                    0.38  
                 9                                    0.53  
                 9                                    0.04  
                 9                                    0.08  
                 9                                    0.04  
                 9                                    0.48  
                 9                                    0.04  
                 9                                    0.04  
                 9                                    0.05  
                 9                                    0.05

$$ \sum_{k}{w_{kpjcstm}} \leq max \ F_{kpjcstm} $$

In [6]:
df3["Suma de Peso [kg/item]"] = df3.groupby(["Puerto de origen", "Puerto de destino", "Transportista", 
                       "Nivel de servicio", "Envío [días]", "Modo de transporte"])["Peso [kg/item]"].transform("sum")

df3["Suma de Peso [kg/item]"] = np.where(df3["Suma de Peso [kg/item]"] <= df3["Peso máximo [kg/item]"], df3["Suma de Peso [kg/item]"], df3["Peso máximo [kg/item]"])

df3[["Puerto de origen", "Puerto de destino", "Transportista", "Nivel de servicio", "Peso [kg/item]"]].set_index(["Puerto de origen", "Puerto de destino"]).sample(10).round(2)

Transportista Nivel de servicio  \
Puerto de origen Puerto de destino                                   
4                9                        V444_1               DTD   
                 9                        V444_0               DTP   
                 9                        V444_1               DTD   
                 9                        V444_0               DTP   
                 9                        V444_0               DTP   
                 9                        V444_1               DTD   
                 9                        V444_0               DTP   
                 9                        V444_1               DTD   
                 9                        V444_1               DTD   
                 9                        V444_0               DTP   

                                    Peso [kg/item]  
Puerto de origen Puerto de destino                  
4                9                           93.78  
                 9                            1.03  
                 9                            8.88  
                 9                           58.74  
                 9                            0.04  
                 9                           88.84  
                 9                            3.42  
                 9                            1.14  
                 9                            7.20  
                 9                            0.37

$$ Si \ s = CRF \ \Rightarrow \ Y_{kpjcstm} = 0 $$

$$ Si \ s \neq CRF, \ m = Tierra \ \Rightarrow \ Y_{kpjcstm} = \frac{w_{kpjcstm}}{\sum_{k}{w_{kpjcstm}}}*R_{kpjcstm} $$

$$ Si \ s \neq CRF, \ m \neq Tierra \ \Rightarrow \ Y_{kpjcstm} = w_{kpjcstm}*R_{kpjcstm} $$

$$ Si \ Y_{kpjcstm} < M_{kpjcstm} \ \Rightarrow \ Y_{kpjcstm} = M_{kpjcstm} $$

In [7]:
a = df3["Peso [kg/item]"] / df3["Suma de Peso [kg/item]"] * df3["Tasa de flete [USD/kg]"]
b = df3["Peso [kg/item]"] * df3["Tasa de flete [USD/kg]"]
c = df3["Costo mínimo [USD/kg]"]

df3["Costo de transporte [USD]"] = np.where(df3["Nivel de servicio"] == "CRF", 0, 
                                            np.where(df3["Modo de transporte"] == "Tierra", np.where(a < c, a, c), np.where(b < c, b, c)))

df3[["Puerto de origen", "Puerto de destino", "Transportista", "Nivel de servicio", "Costo de transporte [USD]"]].set_index(["Puerto de origen", "Puerto de destino"]).sample(10).round(2)

Transportista Nivel de servicio  \
Puerto de origen Puerto de destino                                   
4                9                        V444_1               DTD   
                 9                        V444_1               DTD   
                 9                        V444_0               DTP   
                 9                        V444_0               DTP   
                 9                        V444_1               DTD   
                 9                        V444_1               DTD   
                 9                        V444_1               DTD   
                 9                        V444_0               DTP   
                 9                        V444_0               DTP   
                 9                        V444_1               DTD   

                                    Costo de transporte [USD]  
Puerto de origen Puerto de destino                             
4                9                                       0.02  
                 9                                       0.19  
                 9                                       0.19  
                 9                                       1.02  
                 9                                       1.20  
                 9                                       1.44  
                 9                                       1.44  
                 9                                       1.50  
                 9                                       0.02  
                 9                                       2.31

$$ Y_{pj} = \sum_{cstm}{Y_{pjcstm}} $$

In [8]:
g3 = df3.groupby(by = ["Puerto de origen", "Puerto de destino"]).agg({"Costo de transporte [USD]": np.sum}).round(2)
g3

,,Costo de transporte [USD]
Puerto de origen,Puerto de destino,
4,9,111527.17


### Costos Totales

$$ Z_{ipj} = min \ (X_{i} + Y_{pj}) $$

*¿Qué conjunto Planta, Puerto de origen y Puerto de destino tiene la menor Suma de Costo Total?*

In [17]:
index = df3.groupby(by = ["Planta", "Puerto de origen", "Puerto de destino"]).groups
index = pd.DataFrame(data = index.keys(), columns = ["Planta", "Puerto de origen", "Puerto de destino"])

index


,Planta,Puerto de origen,Puerto de destino
0,3,4,9
1,8,4,9
2,9,4,9
3,12,4,9
4,13,4,9


In [12]:
g4 = pd.merge(g1, g3, how = "cross")
g4["Costo total [USD]"] = g4["Costo [USD/pedido]"] + g4["Costo de transporte [USD]"]

g4

,Costo [USD/pedido],Costo de transporte [USD],Costo total [USD]
0,2.498690e+08,111527.17,2.499805e+08
1,3.039130e+03,111527.17,1.145663e+05
2,2.419147e+06,111527.17,2.530674e+06
3,1.954737e+06,111527.17,2.066264e+06
4,9.781107e+05,111527.17,1.089638e+06
5,3.788345e+05,111527.17,4.903617e+05


In [14]:
g4.min()

Costo [USD/pedido]             3039.12954
Costo de transporte [USD]    111527.17000
Costo total [USD]            114566.29954
dtype: float64